# Цель проекта:
Провести анализ сервиса вопросов и ответов по программированию.

С помощью SQL посчитаем ключевые метрики сервис-системы вопросов и ответов о программировании.

# Описание таблиц

#### Таблица stackoverflow.posts
Содержит информацию о постах.

      Поле	Описание
      id	- Идентификатор поста, первичный ключ таблицы
      title	- Заголовок поста
      creation_date	- Дата создания поста
      favorites_count	- Число, которое показывает, сколько раз пост добавили в «Закладки»
      last_activity_date	- Дата последнего действия в посте, например комментария
      last_edit_date	- Дата последнего изменения поста
      user_id	- Идентификатор пользователя, который создал пост, внешний ключ к таблице users
      parent_id	- Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
      post_type_id	- Идентификатор типа поста, внешний ключ к таблице post_types
      score	- Количество очков, которое набрал пост
      views_count	- Количество просмотров

#### Таблица stackoverflow.users
Содержит информацию о пользователях.

    Поле	Описание
    id	- Идентификатор пользователя, первичный ключ таблицы
    creation_date	- Дата регистрации пользователя
    display_name	- Имя пользователя
    last_access_date	- Дата последнего входа
    location	- Местоположение
    reputation	- Очки репутации, которые получают за хорошие вопросы и полезные ответы
    views	- Число просмотров профиля пользователя

# Исследовательский анализ.

###### Выведим общую сумму просмотров постов за каждый месяц 2008 года.
Если данных за какой-либо месяц в базе нет, такой месяц пропустим. Результат отсортируем по убыванию общего количества просмотров.

In [ ]:
'''
SELECT
  CAST(DATE_TRUNC('MONTH', creation_date) AS DATE) AS report_dt,
  SUM(views_count) AS total
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY 1
ORDER BY 2 DESC
'''

report_dt	 |  total

2008-09-01	| 452928568

2008-10-01	| 365400138

...

2008-07-01	| 669895


*Видим, что данные отличаются. Возможно, повышенная активность в сентябре и октябре связана с началом учебного года. Малая активность в июле может свидетельствовать о неполноте данных.*

###### Выведим имена самых активных пользователей,
которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, учитывать не будем. Для каждого имени пользователя выведим количество уникальных значений `user_id`. Отсортируем результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
SELECT
  u.display_name,
  COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p
  ON u.id = p.user_id
JOIN stackoverflow.post_types AS pt
  ON p.post_type_id = pt.id
WHERE CAST(p.creation_date AS DATE) BETWEEN CAST(u.creation_date AS DATE) AND (CAST(u.creation_date AS DATE) + INTERVAL '1 month')    AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(*) > 100
ORDER BY u.display_name;
'''

display_name	 | count

1800 INFORMATION	 |  1

Adam Bellaire	| 1

...

Vilx-	| 1

Vinko Vrsalovic	| 1


*Кажется, что одному имени пользователя должен соответствовать один `user_id`. Но это не так: многим популярным именам вроде Alan, Dan или Chris соответствует несколько значений `user_id`. Данные лучше не анализировать по имени, иначе результаты будут некорректными.*

###### Выведиv количество постов за 2008 год по месяцам.
Отберем посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортирем таблицу по значению месяца по убыванию.

In [ ]:
'''
WITH tmp1 AS (
SELECT
  CAST(DATE_TRUNC('month', p.creation_date) AS DATE) AS creation_date,
  p.user_id,
  COUNT(*) AS cnt
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u
    ON u.id = p.user_id
    AND CAST(DATE_TRUNC('month', u.creation_date) AS DATE) = '2008-09-01'
WHERE EXTRACT(YEAR FROM p.creation_date) = 2008
GROUP BY 1,2
)
SELECT
  creation_date,
  SUM(cnt) AS cnt
FROM tmp1
WHERE user_id IN (SELECT DISTINCT user_id
                  FROM stackoverflow.posts
                  WHERE EXTRACT(YEAR FROM creation_date) = 2008
                   AND EXTRACT(MONTH FROM creation_date) = 12)
GROUP BY 1
ORDER BY 1 DESC

'''

creation_date	| cnt

2008-12-01	| 17641

2008-11-01	| 18294

...

2008-09-01	| 24870

2008-08-01	| 32

*В итоговой таблице встречаются аномальные значения: пользователи, зарегистрированные в сентябре, были активны и в августе. Возможно, это ошибка в данных.*

###### Используя данные о постах, выведим несколько полей:

*  идентификатор пользователя, который написал пост;
*  дата создания поста;
*  количество просмотров у текущего поста;
*  сумму просмотров постов автора с накоплением.

Данные в таблице отсортируем по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
SELECT *
FROM(
        SELECT
          user_id,
          creation_date,
          views_count,
          SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) AS total_v
        FROM stackoverflow.posts
     )AS t
ORDER BY user_id, creation_date
'''

user_id	| creation_date	| views_count	| total_v

1	| 2008-07-31 23:41:00	| 480476	| 480476

1	| 2008-07-31 23:55:38	| 136033	| 616509

...

5	| 2008-09-12 16:20:24	| 3839	| 79444

*В теории все расчёты с оконной функцией можно выполнить и без неё. Но размер запроса имеет значение.*

###### Посмотрим, сколько в среднем дней, в период с 1 по 7 декабря 2008 года включительно, пользователи взаимодействовали с платформой?
Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Результат округлим .

In [ ]:
'''
SELECT
  ROUND(AVG(cnt)) AS avg_cnt
FROM (
       SELECT
          user_id,
          COUNT(*) AS cnt
        FROM (
               SELECT DISTINCT
                  user_id,
                  EXTRACT(DAY FROM CAST(creation_date AS DATE)) AS day
                FROM stackoverflow.posts
                WHERE CAST(creation_date AS DATE) BETWEEN '2008-12-01' AND '2008-12-07'
             )t
        GROUP BY 1
     )t
'''

avg_cnt

2

###### На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразим таблицу со следующими полями:

*  номер месяца;
*  количество постов за месяц;
*  процент, который показывает, насколько изменилось
*  количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлим значение процента до двух знаков после запятой.
Помним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведем делимое в тип `numeric`.

In [ ]:
'''
WITH tmp1 AS (
SELECT
  EXTRACT(MONTH FROM CAST(p.creation_date AS DATE)) AS month_dt,
  COUNT(*) AS cnt
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u
      ON p.user_id = u.id
WHERE CAST(p.creation_date AS DATE) BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY 1
)
SELECT *
FROM (
        SELECT
          month_dt,
          cnt,
         -- LAG(cnt, 1, cnt) OVER (ORDER BY month_dt) AS prev_cnt,
          ROUND((cnt - LAG(cnt) OVER (ORDER BY month_dt)) * 1.0 / LAG(cnt, 1, cnt) OVER (ORDER BY month_dt) * 100,2) AS pr
        FROM tmp1
    )t
WHERE month_dt > 8
'''

month_dt	| cnt	| pr

10	| 63102	| -10.33

...

12	| 44592	| -5.07



###### Выгрузим данные активности пользователя, который опубликовал больше всего постов за всё время.
Выведим данные за октябрь 2008 года в таком виде:

 * номер недели;
 * дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
WITH tmp1 AS (
SELECT
  p.user_id,
  COUNT(*)
FROM stackoverflow.posts AS p
JOIN stackoverflow.posts AS u
    ON p.user_id = u.id
GROUP BY p.user_id
ORDER BY 2 DESC
LIMIT 1
)
SELECT
  EXTRACT(WEEK FROM creation_date) AS wk,
  MAX(creation_date) AS creation_date
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('MONTH', creation_date) AS DATE) = '2008-10-01'
  AND user_id IN (SELECT user_id FROM tmp1)
GROUP BY 1
'''

wk	| creation_date

40	| 2008-10-05 09:00:58

41	| 2008-10-12 21:22:23

...

44	| 2008-10-31 22:16:01

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######

In [ ]:
'''

'''

######